In [2]:
!pip install -q langchain dspy-ai crewai google-search-results numexpr sentencepiece plotly

^C
ERROR: Operation cancelled by user

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
%reload_ext autoreload
%autoreload 2

In [6]:
from langchain.agents import AgentExecutor, load_tools
from langchain.chat_models import ChatHuggingFace, ChatOpenAI
from langchain_community.llms import HuggingFaceEndpoint
from langchain_core.tools import Tool
from scripts.modified_calculator import LLMMathChain
from scripts.agents import build_hf_agent, build_openai_agent
from scripts.run_agents import answer_questions

In [163]:
OPENAI_MODEL_ID = "gpt-4-0125-preview"

# DSPy

In [164]:
import dspy

gpt4_llm_dspy = dspy.OpenAI(model=OPENAI_MODEL_ID)
dspy.settings.configure(lm=gpt4_llm_dspy)

### Define tools

In [165]:
llm = ChatOpenAI(mdoel=OPENAI_MODEL_ID)

langchain_math_tool = Tool(
    name="Calculator",
    description="Useful for when you need to answer questions about math.",
    func=LLMMathChain.from_llm(llm=llm).run,
    coroutine=LLMMathChain.from_llm(llm=llm).arun,
)
TOOLS = load_tools(["serpapi", "llm-math"], llm=llm)
langchain_serpapi_tool = TOOLS[0]


class DSPyMathTool:
    name = "calculator"
    input_variable = "math formula"
    desc = "Useful for when you need to answer questions about math."

    def __init__(self, k=3):
        pass

    def __call__(self, *args, **kwargs):
        return langchain_math_tool.run(*args, **kwargs)


class DSPySerpAPITool:
    name = "search"
    input_variable = "search input"
    desc = langchain_serpapi_tool.description

    def __init__(self, k=3):
        pass

    def __call__(self, *args, **kwargs):
        return langchain_serpapi_tool.run(*args, **kwargs)

In [166]:
import datetime
import dspy


class GiveTime:
    name = "GiveTime"
    input_variable = "empty"
    desc = "takes an empty string and returns the current local time"

    def __init__(self, k=3):
        pass

    def __call__(self, *args, **kwargs):
        return datetime.datetime.now().strftime("%H:%M:%S")

In [167]:
math_tool = DSPyMathTool()
serpapi_tool = DSPySerpAPITool()
agent_dspy = dspy.ReAct("question -> answer", tools=[math_tool, serpapi_tool])

In [168]:
async def call_dspy(agent, question: str) -> str:
    response = {"output": agent(question=question).answer, "intermediate_steps": []}
    return response

In [ ]:
await call_dspy(agent_dspy, "what is 2+2")

Backing off 0.1 seconds after 1 tries calling function <function GPT3.request at 0x7f1d8e981480> with kwargs {}
Backing off 1.1 seconds after 2 tries calling function <function GPT3.request at 0x7f1d8e981480> with kwargs {}
Backing off 0.8 seconds after 3 tries calling function <function GPT3.request at 0x7f1d8e981480> with kwargs {}
Backing off 7.6 seconds after 4 tries calling function <function GPT3.request at 0x7f1d8e981480> with kwargs {}
Backing off 9.7 seconds after 5 tries calling function <function GPT3.request at 0x7f1d8e981480> with kwargs {}
Backing off 28.2 seconds after 6 tries calling function <function GPT3.request at 0x7f1d8e981480> with kwargs {}
Backing off 28.1 seconds after 7 tries calling function <function GPT3.request at 0x7f1d8e981480> with kwargs {}
Backing off 90.6 seconds after 8 tries calling function <function GPT3.request at 0x7f1d8e981480> with kwargs {}
Backing off 93.6 seconds after 9 tries calling function <function GPT3.request at 0x7f1d8e981480> wit

### Compile DSPy

In [11]:
import datasets

math_dataset = datasets.load_dataset("gsm8k", "main")["train"]


def build_dspy_trainset(dataset):
    dspy_dataset = []
    for i in range(len(dataset)):
        dspy_dataset.append(
            dspy.Example(
                question=dataset["question"][i],
                answer=dataset["answer"][i],
            ).with_inputs("question")
        )
    return dspy_dataset


# from dspy.teleprompt import BootstrapFewShot

# teleprompter = BootstrapFewShot(metric=validate_with_llm_judge, max_rounds=5)
# compiled_rag = teleprompter.compile(
#     agent,
#     trainset=build_dspy_trainset(math_dataset[:20]),
#     valset=build_dspy_trainset(math_dataset[20:50]),
# )

# Benchmark different frameworks

In [169]:
import datasets

math_dataset = (
    datasets.load_dataset("reasoning-machines/gsm-hard", "main")["train"]
    .shuffle(seed=42)
    .select(range(30))
)
math_dataset = math_dataset.rename_column("input", "question")
math_dataset = math_dataset.rename_column("target", "answer")

task_column = ["GSM8K-hard"] * len(math_dataset)
math_dataset = math_dataset.add_column("task", task_column).select_columns(
    ["question", "answer", "task"]
)
eval_dataset = math_dataset

OUTPUT_DIR = "output_gsm8k-hard"

/home/ubuntu/venv/agents/lib/python3.10/site-packages/datasets/table.py:1387: FutureWarning:

promote has been superseded by promote_options='default'.



### DSPy

In [14]:
# DSPy
await answer_questions(
    eval_dataset,
    agent_dspy,
    "dspy_agent",
    output_folder=OUTPUT_DIR,
    agent_call_function=call_dspy,
)

NameError: name 'answer_questions' is not defined

### Vanilla Agent

In [170]:
prompt = "Answer the question below. You can think out loud, but remind to ALWAYS use the exact characters `Final Answer:` when you provide a definitive answer. Question:"


async def call_vanilla_agent(agent, question: str):
    output = agent.invoke(prompt + question).content
    if "Final Answer:" in output:
        output = output.split("Final Answer:")[1]
    response = {"output": output, "intermediate_steps": []}
    return response

In [15]:
MODEL_NAME = "zephyr-7b"
url = "https://gw38hkbdke51vu7l.us-east-1.aws.endpoints.huggingface.cloud"

In [173]:
OPENAI_MODEL_ID = "gpt-4-0125-preview"

In [174]:
MODEL_NAME = "gpt4"
llm_agent = ChatOpenAI(model=OPENAI_MODEL_ID)
# endpoint = HuggingFaceEndpoint(endpoint_url=url)
# llm_agent = ChatHuggingFace(llm=endpoint)


await answer_questions(
    eval_dataset,
    llm_agent,
    f"{MODEL_NAME}_notools",
    output_folder=OUTPUT_DIR,
    agent_call_function=call_vanilla_agent,
)

100%|██████████| 30/30 [06:07<00:00, 12.24s/it]


[{'agent_name': 'gpt4_notools',
  'question': 'Charlie wants to sell beeswax candles.  For every pound of beeswax, he can make 10 tapered candles.  One pound of beeswax and the wicks cost $10.00 in supplies.   If he sells each candle for $2.00 each, what is his net profit if he makes and sells 1080379 candles?',
  'gt_answer': 1080379.0,
  'prediction': " Charlie's net profit if he makes and sells 1,080,379 candles would be \\$1,080,378.",
  'intermediate_steps': [],
  'parsing_error': False,
  'iteration_limit_exceeded': False,
  'agent_error': None,
  'start_time': '2024-02-12 18:01:20',
  'end_time': '2024-02-12 18:01:38',
  'task': 'GSM8K-hard'},
 {'agent_name': 'gpt4_notools',
  'question': 'Melissa is summoned to jury duty. She spends 6 hours a day for 3 days listening to a court case. If Melissa is paid $557654 per day but also has to pay $3 for parking each day, how much jury pay does she make per hour after expenses?',
  'gt_answer': 92941.8333333333,
  'prediction': ' $92,941

### LangChain

In [ ]:
from scripts.agents import build_openai_agent, build_hf_agent

langchain_agent = build_openai_agent(model_id=OPENAI_MODEL_ID)
MODEL_NAME = "gpt-3.5"
# langchain_agent = build_hf_agent(hf_endpoint_url=url)
await answer_questions(
    eval_dataset,
    langchain_agent,
    f"{MODEL_NAME}_langchain_nosearch",
    output_folder=OUTPUT_DIR,
)

### CrewAI

In [34]:
from crewai import Agent, Task, Crew, Process

langchain_agent = build_openai_agent(model_id=OPENAI_MODEL_ID)

llm_agent = ChatOpenAI(model=OPENAI_MODEL_ID)
MODEL_NAME = "gpt-3.5"

langchain_math_tool = Tool(
    name="Calculator",
    description="Useful for when you need to answer questions about math.",
    func=LLMMathChain.from_llm(llm=llm_agent).run,
    coroutine=LLMMathChain.from_llm(llm=llm_agent).arun,
)
TOOLS = load_tools(["serpapi", "llm-math"], llm=llm_agent)
langchain_serpapi_tool = TOOLS[0]


async def call_crew_sequential(agent, question: str):
    # Creating a senior researcher agent
    researcher = Agent(
        role="Senior Researcher",
        goal=f"Solve this question: {question}",
        verbose=True,
        backstory="""Driven by curiosity, you like solving math problems""",
        llm=agent,
    )

    # Research task for identifying AI trends
    solving_task = Task(
        description=f"""Solve the given question: {question}. Think step-by-step. Remember to preface your final answer with 'Final Answer:'.""",
        expected_output="A short answer",
        max_inter=5,
        tools=[langchain_math_tool, langchain_serpapi_tool],
        agent=researcher,
    )

    # Forming the tech-focused crew
    crew = Crew(
        agents=[researcher],
        tasks=[solving_task],
        process=Process.sequential,  # Sequential task execution
    )
    output = crew.kickoff()

    if "Final Answer:" in output:
        output = output.split("Final Answer:")[1]
    response = {"output": output, "intermediate_steps": []}
    return response


# llm_gpt = ChatOpenAI(model=OPENAI_MODEL_ID, temperature=0)

await answer_questions(
    eval_dataset,
    llm_agent,
    f"{MODEL_NAME}_crewai_solver_with_tools",
    output_folder=OUTPUT_DIR,
    agent_call_function=call_crew_sequential,
)

Removing search


  0%|          | 0/30 [00:00<?, ?it/s]



> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: Calculate the total cost of supplies for 1080379 candles and the total revenue from selling those candlesDo I need to use a tool? Yes
Action: Calculator
Action Input: Calculate the total cost of supplies for 1080379 candles and the total revenue from selling those candles 

I just used the Calculator tool with input {'tool_input': {'type': 'string', 'value': 'Calculate the total cost of supplies for 1080379 candles and the total revenue from selling those candles'}}. So I already know the result of that and don't need to use it again now. 
I could give my final answer if I'm ready, using exaclty the expected format bellow: 
```
Thought: Do I need to use a tool? No
Final Answer: [your response here]```


Do I need to use a tool? No
Final Answer: The total cost of supplies for 1080379 candles would be $10.00 per pound of beeswax and wicks. Since Charlie can make 10 candles f

  3%|▎         | 1/30 [00:14<06:51, 14.18s/it]



> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: Calculate the total earnings for Melissa and the total parking expenses for the 3 daysDo I need to use a tool? Yes
Action: Calculator
Action Input: Calculate the total earnings for Melissa and the total parking expenses for the 3 days manually 

I just used the Calculator tool with input {'tool_input': 'Calculate the total earnings for Melissa and the total parking expenses for the 3 days manually'}. So I already know the result of that and don't need to use it again now. 
I could give my final answer if I'm ready, using exaclty the expected format bellow: 
```
Thought: Do I need to use a tool? No
Final Answer: [your response here]```


Do I need to use a tool? No
Final Answer: Melissa makes $557,654 per day, but she has to pay $3 for parking each day. In total for 3 days, she earns $1,673,962. After deducting parking expenses ($3 * 3 days = $9), her net earnings are $1,67

...



> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: Calculate the total number of wooden action figures Peter bought from Sunday through Wednesday and the total number of plastic action figures he will buy for the rest of the week. 

I just used the Calculator tool with input {'tool_input': 'Calculate the total number of wooden action figures Peter bought from Sunday through Wednesday and the total number of plastic action figures he will buy for the rest of the week.'}. So I already know the result of that and don't need to use it again now. 
I could give my final answer if I'm ready, using exaclty the expected format bellow: 
```
Thought: Do I need to use a tool? No
Final Answer: [your response here]```


Do I need to use a tool? No
Final Answer: Peter will have a total of 1,334,304 action figures by the end of the week.

> Finished chain.


 97%|█████████▋| 29/30 [02:14<00:04,  4.22s/it]



> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? No

Final Answer: The average total distance spat is 5,878,712 feet.

> Finished chain.


100%|██████████| 30/30 [02:16<00:00,  4.56s/it]


[{'agent_name': 'gpt-3.5_crewai_solver_with_tools',
  'question': 'Charlie wants to sell beeswax candles.  For every pound of beeswax, he can make 10 tapered candles.  One pound of beeswax and the wicks cost $10.00 in supplies.   If he sells each candle for $2.00 each, what is his net profit if he makes and sells 1080379 candles?',
  'gt_answer': 1080379.0,
  'prediction': "Charlie's net profit from making and selling 1080379 candles would be $2,052,720.10.",
  'intermediate_steps': [],
  'parsing_error': False,
  'iteration_limit_exceeded': False,
  'agent_error': None,
  'start_time': '2024-02-12 17:04:20',
  'end_time': '2024-02-12 17:04:34',
  'task': 'GSM8K-hard'},
 {'agent_name': 'gpt-3.5_crewai_solver_with_tools',
  'question': 'Melissa is summoned to jury duty. She spends 6 hours a day for 3 days listening to a court case. If Melissa is paid $557654 per day but also has to pay $3 for parking each day, how much jury pay does she make per hour after expenses?',
  'gt_answer': 929

In [28]:
from crewai import Agent, Task, Crew, Process

langchain_math_tool = Tool(
    name="Calculator",
    description="Useful for when you need to answer questions about math.",
    func=LLMMathChain.from_llm(llm=llm_agent).run,
    coroutine=LLMMathChain.from_llm(llm=llm_agent).arun,
)
TOOLS = load_tools(["serpapi", "llm-math"], llm=llm_agent)
langchain_serpapi_tool = TOOLS[0]


async def call_crew_sequential(agent, question: str):
    # Creating a senior researcher agent
    researcher = Agent(
        role="Senior Researcher",
        goal=f"Solve this question: {question}",
        verbose=True,
        backstory="""Driven by curiosity, you like solving math problems""",
        llm="agent",
    )

    # Creating a writer agent
    veryfier = Agent(
        role="Senior Verifyer",
        goal=f"Check results of others for this question: {question}",
        verbose=True,
        backstory="""You have a flair to detect errors in math problems and correct them.""",
        llm=agent,
    )
    # Research task for identifying AI trends
    solving_task = Task(
        description=f"""Solve the given question: {question}. Think step-by-step.""",
        expected_output="A short answer",
        max_inter=3,
        tools=[langchain_math_tool, langchain_serpapi_tool],
        agent=researcher,
    )

    # Writing task based on research findings
    verification_task = Task(
        description=f"""You have to answer {question}, then provide a final answer.
        Think step-by-step to correct any errors. Remember to preface your final answer with 'Final Answer:'. In your final answer, do not talk about wether the previous answer was correct or not. Just provide the corrected answer.
        """,
        expected_output=f"The corrected answer to the question {question}",
        tools=[langchain_math_tool, langchain_serpapi_tool],
        agent=veryfier,
    )
    # Forming the tech-focused crew
    crew = Crew(
        agents=[researcher, veryfier],
        tasks=[solving_task, verification_task],
        process=Process.sequential,  # Sequential task execution
    )
    output = crew.kickoff()

    if "Final Answer:" in output:
        output = output.split("Final Answer:")[1]
    response = {"output": output, "intermediate_steps": []}
    return response


# llm_gpt = ChatOpenAI(model=OPENAI_MODEL_ID, temperature=0)

await answer_questions(
    eval_dataset,
    llm_agent,
    f"{MODEL_NAME}_crewai_sequential",
    output_folder=OUTPUT_DIR,
    agent_call_function=call_crew_sequential,
)

NameError: name 'llm_agent' is not defined

# Evaluate results

In [175]:
%reload_ext autoreload
%autoreload 2

In [187]:
import pandas as pd
import glob

answer_file_path = "output_gsm8k-hard/answers.jsonl"

result_df = pd.concat([pd.read_json(f) for f in glob.glob("output_gsm8k-hard/*.json")])
result_df = result_df.drop(columns=["start_time", "end_time"])
result_df.to_json(answer_file_path, lines=True, orient="records")

In [188]:
answer_df = pd.read_json(answer_file_path, lines=True)
answers = answer_df.to_dict(orient="records")
answers = answers

In [189]:
answer_df["agent_name"].value_counts()

agent_name
zephyr-7b_langchain_nosearch     30
gpt4_crewai_solver_with_tools    30
gpt3.5_langchain2                30
gpt3.5_agent_notools2            30
gpt-3.5_langchain_nosearch       30
gpt4_notools                     30
gpt3.5_agent_notools             30
zephyr-7b_langchain              30
zephyr-7b_agent_notools          30
zephyr-7b_crewai_sequential      11
gpt4_crewai_sequential            5
dspy_agent                        4
Name: count, dtype: int64

# Score exact match

In [190]:
results_df = pd.read_json(answer_file_path, lines=True)
results_df["agent_name"].value_counts()

agent_name
zephyr-7b_langchain_nosearch     30
gpt4_crewai_solver_with_tools    30
gpt3.5_langchain2                30
gpt3.5_agent_notools2            30
gpt-3.5_langchain_nosearch       30
gpt4_notools                     30
gpt3.5_agent_notools             30
zephyr-7b_langchain              30
zephyr-7b_agent_notools          30
zephyr-7b_crewai_sequential      11
gpt4_crewai_sequential            5
dspy_agent                        4
Name: count, dtype: int64

In [191]:
import numpy as np
import re


def find_number(string):
    try:
        found_strings = [el.strip() for el in re.findall(r"(?:[,\d]+.?\d*)", string)]

        found_strings = [
            "".join(ch for ch in el if (ch.isalnum() or ch == "."))
            for el in found_strings
            if el[0].isdigit() or el[0] == "."
        ]
        found_strings = [el for el in found_strings if len(el) > 0]

        found_string = found_strings[0]
        return float(found_string)
    except Exception as e:
        return 0


results_df["extracted_answer"] = results_df["prediction"].apply(find_number)
results_df["exact_match"] = np.isclose(
    results_df["extracted_answer"], results_df["gt_answer"], rtol=0.1
)

In [192]:
aggregate = results_df.groupby("agent_name")[["exact_match"]].mean().reset_index()

In [193]:
import plotly.express as px

fig = px.bar(aggregate, x="agent_name", y=["exact_match"])
fig.update_layout(barmode="group", width=1000)

### GPT4 eval

In [51]:
from langchain.chat_models import ChatOpenAI

eval_chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
eval_model_name = "GPT4"

In [102]:
from scripts.evaluation import evaluate_answers
from scripts.prompts import EVALUATION_PROMPT_TEMPLATE

output_file_path = "output_gsm8k-hard/evaluation.jsonl"

run_evaluation = False
if run_evaluation:
    results = await evaluate_answers(
        answers,
        eval_chat_model,
        "GPT4",
        EVALUATION_PROMPT_TEMPLATE,
        output_file_path=output_file_path,
    )

    print("Evaluation is complete!")

Previous evaluations:
Launching evaluation for 0 examples...


0it [00:00, ?it/s]

Evaluation is complete!


In [105]:
def interpret_score(eval_score):
    try:
        return (float(eval_score) - 1) / 4
    except:
        return 0


results_df["eval_score_GPT4"] = results_df["eval_score_GPT4"].apply(interpret_score)
aggregate = results_df.groupby("agent_name")["eval_score_GPT4"].mean()
aggregate

agent_name
dspy_agent                       0.000000
gpt-3.5_langchain_nosearch       0.483333
gpt3.5_agent_notools             0.558333
gpt3.5_agent_notools2            0.591667
gpt3.5_langchain2                0.466667
gpt4_crewai_sequential           0.950000
gpt4_crewai_solver_with_tools    0.300000
zephyr-7b_agent_notools          0.175000
zephyr-7b_crewai_sequential      0.568182
zephyr-7b_langchain              0.025000
zephyr-7b_langchain_nosearch     0.008333
Name: eval_score_GPT4, dtype: float64